In [1]:
!pip install pymupdf pillow

In [2]:
import fitz  # PyMuPDF
import os
import json

# Upload the PDF
from google.colab import files
uploaded = files.upload()

pdf_path = list(uploaded.keys())[0]
output_image_dir = "pdf_images"
os.makedirs(output_image_dir, exist_ok=True)

doc = fitz.open(pdf_path)
extracted_content = []

for page_num in range(len(doc)):
    page = doc[page_num]
    text = page.get_text()
    images_info = []

    for img_index, img in enumerate(page.get_images(full=True)):
        xref = img[0]
        base_image = doc.extract_image(xref)
        image_bytes = base_image["image"]
        image_ext = base_image["ext"]
        image_filename = f"page{page_num+1}_image{img_index+1}.{image_ext}"
        image_path = os.path.join(output_image_dir, image_filename)
        
        with open(image_path, "wb") as f:
            f.write(image_bytes)
        
        images_info.append(image_path)
    
    extracted_content.append({
        "page": page_num + 1,
        "text": text.strip(),
        "images": images_info
    })

# Save the structured content
with open("structured_output.json", "w") as f:
    json.dump(extracted_content, f, indent=2)

In [3]:
!pip install transformers torch torchvision

In [4]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

def generate_caption(image_path):
    raw_image = Image.open(image_path).convert('RGB')
    inputs = processor(raw_image, return_tensors="pt")
    out = model.generate(**inputs)
    return processor.decode(out[0], skip_special_tokens=True)

In [5]:
import glob

image_paths = glob.glob("pdf_images/*.png")
qa_list = []

for img in image_paths:
    caption = generate_caption(img)
    question = f"What is shown in the image: '{caption}'?"
    qa_list.append({
        "question": question,
        "images": img,
        "option_images": []  # You can add if found later
    })

with open("ai_generated_questions.json", "w") as f:
    json.dump(qa_list, f, indent=2)

In [6]:
import glob

image_paths = glob.glob("pdf_images/*.png")
qa_list = []

for img in image_paths:
    caption = generate_caption(img)
    question = f"What is shown in the image: '{caption}'?"
    qa_list.append({
        "question": question,
        "images": img,
        "option_images": []  # You can add if found later
    })

with open("ai_generated_questions.json", "w") as f:
    json.dump(qa_list, f, indent=2)

In [7]:
import glob

image_paths = glob.glob("pdf_images/*.png")
qa_list = []

for img in image_paths:
    caption = generate_caption(img)
    question = f"What is shown in the image: '{caption}'?"
    qa_list.append({
        "question": question,
        "images": img,
        "option_images": []  # You can add if found later
    })

with open("ai_generated_questions.json", "w") as f:
    json.dump(qa_list, f, indent=2)

In [8]:
with open("ai_generated_questions.json", "r") as f:
    data = json.load(f)

pprint(data[:5])  # View first 5 entries

In [9]:
with open("ai_generated_questions.json", "r") as f:
    data = json.load(f)

from pprint import pprint

pprint(data[:5])  # View first 5 entries

In [10]:
from IPython.display import HTML

HTML('''
  <a href="javascript:window.print()" target="_blank">
    <button style="padding:10px 20px;font-size:16px;margin-top:10px">
      📄 Save as PDF
    </button>
  </a>
''')